In [ ]:
from ultralytics import YOLO
import cv2
import os
import glob

# =====================================================
# 1️⃣ Kiểm tra dataset và tạo báo cáo
# =====================================================
def check_dataset(dataset_path="datasets"):
    """Kiểm tra số lượng images/labels và phát hiện file thiếu"""
    print("=== Kiểm tra dataset ===")
    for split in ["train", "val", "test"]:
        img_dir = os.path.join(dataset_path, "images", split)
        lbl_dir = os.path.join(dataset_path, "labels", split)
        
        if not os.path.exists(img_dir):
            print(f"⚠️  Không tìm thấy: {img_dir}")
            continue
            
        images = set([os.path.splitext(f)[0] for f in os.listdir(img_dir)])
        labels = set([os.path.splitext(f)[0] for f in os.listdir(lbl_dir)])
        
        missing_labels = images - labels
        missing_images = labels - images
        
        print(f"\n{split.upper()}:")
        print(f"  ✓ {len(images)} images, {len(labels)} labels")
        if missing_labels:
            print(f"  ⚠️  {len(missing_labels)} ảnh thiếu label")
        if missing_images:
            print(f"  ⚠️  {len(missing_images)} label thừa (không có ảnh)")

check_dataset()

# =====================================================
# 2️⃣ Tạo file YAML với absolute path
# =====================================================
yaml_content = f"""
path: {os.path.abspath('datasets')}
train: images/train
val: images/val
test: images/test

names:
  0: crop
  1: weed
"""

with open("weed.yaml", "w") as f:
    f.write(yaml_content)

print("\n✓ File weed.yaml đã tạo với absolute path!")

# =====================================================
# 3️⃣ Huấn luyện mô hình (có thể tắt để test nhanh)
# =====================================================
TRAIN_MODEL = True  # Đổi thành False để skip training

if TRAIN_MODEL:
    print("\n=== Bắt đầu huấn luyện ===")
    model = YOLO("yolov8n.pt")
    
    model.train(
        data="weed.yaml",
        epochs=50,
        imgsz=640,
        batch=8,
        device=0,  # 0=GPU, 'cpu'=CPU
        project="weed_detection",
        name="yolov8_weed",
        exist_ok=True,
        patience=10,  # Early stopping sau 10 epochs không cải thiện
        save=True,
        verbose=True
    )
    
    best_model_path = "weed_detection/yolov8_weed/weights/best.pt"
else:
    # Dùng model đã train sẵn
    best_model_path = "weed_detection/yolov8_weed/weights/best.pt"
    if not os.path.exists(best_model_path):
        print("⚠️  Không tìm thấy model đã train. Dùng pretrained model.")
        best_model_path = "yolov8n.pt"

# =====================================================
# 4️⃣ Đánh giá mô hình trên validation và test set
# =====================================================
model = YOLO(best_model_path)

print("\n=== Đánh giá trên VALIDATION set ===")
val_metrics = model.val(data="weed.yaml", split="val")
print(f"mAP50: {val_metrics.box.map50:.3f}")
print(f"mAP50-95: {val_metrics.box.map:.3f}")

print("\n=== Đánh giá trên TEST set ===")
test_metrics = model.val(data="weed.yaml", split="test")
print(f"mAP50: {test_metrics.box.map50:.3f}")
print(f"mAP50-95: {test_metrics.box.map:.3f}")

# =====================================================
# 5️⃣ Dự đoán trên ảnh test với error handling
# =====================================================
def predict_on_test_images(model, test_dir="datasets/images/test", max_images=5):
    """Dự đoán trên tối đa N ảnh test và hiển thị kết quả"""
    test_images = glob.glob(os.path.join(test_dir, "*.jpg"))
    
    if not test_images:
        print(f"⚠️  Không tìm thấy ảnh test trong {test_dir}")
        return
    
    print(f"\n=== Dự đoán trên {min(len(test_images), max_images)} ảnh test ===")
    
    for img_path in test_images[:max_images]:
        print(f"\n📷 {os.path.basename(img_path)}")
        results = model.predict(img_path, conf=0.5, verbose=False)
        
        # In thông tin bbox
        for r in results:
            if len(r.boxes) == 0:
                print("  ➜ Không phát hiện object")
            else:
                for box in r.boxes:
                    cls = int(box.cls[0])
                    cls_name = "crop" if cls == 0 else "weed"
                    conf = float(box.conf[0])
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    print(f"  ➜ {cls_name.upper()} | Conf: {conf:.2f} | BBox: [{x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}]")
        
        # Lưu ảnh có bbox (không hiển thị tự động)
        results[0].save(filename=f"output_{os.path.basename(img_path)}")
    
    print(f"\n✓ Ảnh kết quả đã lưu với tên 'output_*.jpg'")

predict_on_test_images(model, max_images=5)

# =====================================================
# 6️⃣ Real-time detection từ webcam (tuỳ chọn)
# =====================================================
def webcam_detection(model):
    """Phát hiện real-time từ webcam với error handling"""
    print("\n=== Bắt đầu webcam detection (nhấn 'q' để thoát) ===")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("⚠️  Không thể mở webcam. Vui lòng kiểm tra kết nối.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️  Lỗi đọc frame từ webcam")
            break
        
        results = model.predict(frame, conf=0.5, verbose=False)
        annotated_frame = results[0].plot()
        
        cv2.imshow("Weed Detection - Real-time", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("✓ Đã đóng webcam")

# Uncomment dòng dưới để chạy webcam detection
# webcam_detection(model)

In [1]:
from ultralytics import YOLO
import cv2
import os
import glob

# =====================================================
# CẤU HÌNH DATASET
# =====================================================
DATASET_ROOT = "C:\\computervision\\dataset"  # Thư mục chứa train/, valid/, test/
USE_ROBOFLOW_YAML = True  # True = dùng data.yaml có sẵn, False = tạo mới

# =====================================================
# 1️⃣ Kiểm tra dataset và tạo báo cáo
# =====================================================
def check_dataset(dataset_path=DATASET_ROOT):
    """Kiểm tra số lượng images/labels và phát hiện file thiếu"""
    print("=== Kiểm tra dataset ===")
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, split, "images")
        lbl_dir = os.path.join(dataset_path, split, "labels")
        
        if not os.path.exists(img_dir):
            print(f"⚠️  Không tìm thấy: {img_dir}")
            continue
            
        images = set([os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
        labels = set([os.path.splitext(f)[0] for f in os.listdir(lbl_dir) if f.endswith('.txt')])
        
        missing_labels = images - labels
        missing_images = labels - images
        
        print(f"\n{split.upper()}:")
        print(f"  ✓ {len(images)} images, {len(labels)} labels")
        if missing_labels:
            print(f"  ⚠️  {len(missing_labels)} ảnh thiếu label: {list(missing_labels)[:5]}")
        if missing_images:
            print(f"  ⚠️  {len(missing_images)} label thừa: {list(missing_images)[:5]}")

check_dataset()

# =====================================================
# 2️⃣ Chuẩn bị file YAML
# =====================================================
if USE_ROBOFLOW_YAML and os.path.exists(os.path.join(DATASET_ROOT, "data.yaml")):
    yaml_path = os.path.join(DATASET_ROOT, "data.yaml")
    print(f"\n✓ Sử dụng data.yaml có sẵn từ Roboflow: {yaml_path}")
    
    # Đọc và hiển thị nội dung
    with open(yaml_path, "r") as f:
        print("\nNội dung data.yaml:")
        print(f.read())
else:
    # Tạo file YAML mới
    yaml_path = "weed_custom.yaml"
    yaml_content = f"""path: {os.path.abspath(DATASET_ROOT)}
train: train/images
val: valid/images
test: test/images

names:
  0: crop
  1: weed
"""
    
    with open(yaml_path, "w") as f:
        f.write(yaml_content)
    
    print(f"\n✓ Đã tạo file YAML mới: {yaml_path}")

# =====================================================
# 3️⃣ Huấn luyện mô hình
# =====================================================
TRAIN_MODEL = True  # Đổi thành False để skip training
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 8

if TRAIN_MODEL:
    print("\n=== Bắt đầu huấn luyện ===")
    model = YOLO("yolov8n.pt")  # yolov8s.pt cho accuracy cao hơn
    
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=0,  # 0=GPU, 'cpu'=CPU, [0,1]=multi-GPU
        project="weed_detection",
        name="yolov8_weed",
        exist_ok=True,
        patience=15,  # Early stopping
        save=True,
        plots=True,  # Lưu confusion matrix, F1 curve...
        verbose=True
    )
    
    best_model_path = "weed_detection/yolov8_weed/weights/best.pt"
    print(f"\n✓ Model tốt nhất đã lưu tại: {best_model_path}")
else:
    # Dùng model đã train sẵn
    best_model_path = "weed_detection/yolov8_weed/weights/best.pt"
    if not os.path.exists(best_model_path):
        print("⚠️  Không tìm thấy model đã train. Sử dụng pretrained yolov8n.pt")
        best_model_path = "yolov8n.pt"

# =====================================================
# 4️⃣ Đánh giá mô hình
# =====================================================
model = YOLO(best_model_path)

print("\n" + "="*60)
print("ĐÁNH GIÁ MÔ HÌNH TRÊN VALIDATION SET")
print("="*60)
val_metrics = model.val(data=yaml_path, split="val")
print(f"\n📊 Kết quả Validation:")
print(f"   mAP50:    {val_metrics.box.map50:.3f}")
print(f"   mAP50-95: {val_metrics.box.map:.3f}")
print(f"   Precision: {val_metrics.box.mp:.3f}")
print(f"   Recall:    {val_metrics.box.mr:.3f}")

print("\n" + "="*60)
print("ĐÁNH GIÁ MÔ HÌNH TRÊN TEST SET")
print("="*60)
test_metrics = model.val(data=yaml_path, split="test")
print(f"\n📊 Kết quả Test:")
print(f"   mAP50:    {test_metrics.box.map50:.3f}")
print(f"   mAP50-95: {test_metrics.box.map:.3f}")
print(f"   Precision: {test_metrics.box.mp:.3f}")
print(f"   Recall:    {test_metrics.box.mr:.3f}")

# =====================================================
# 5️⃣ Dự đoán trên ảnh test
# =====================================================
def predict_on_test_images(model, test_dir=None, max_images=5, conf=0.25):
    """Dự đoán trên ảnh test và lưu kết quả"""
    if test_dir is None:
        test_dir = os.path.join(DATASET_ROOT, "test", "images")
    
    test_images = glob.glob(os.path.join(test_dir, "*.jpg")) + \
                  glob.glob(os.path.join(test_dir, "*.png"))
    
    if not test_images:
        print(f"⚠️  Không tìm thấy ảnh test trong {test_dir}")
        return
    
    print(f"\n{'='*60}")
    print(f"DỰ ĐOÁN TRÊN {min(len(test_images), max_images)} ẢNH TEST")
    print(f"{'='*60}")
    
    # Tạo thư mục output
    output_dir = "predictions"
    os.makedirs(output_dir, exist_ok=True)
    
    for i, img_path in enumerate(test_images[:max_images], 1):
        print(f"\n[{i}/{min(len(test_images), max_images)}] 📷 {os.path.basename(img_path)}")
        results = model.predict(img_path, conf=conf, verbose=False)
        
        # Đếm số lượng từng class
        crop_count = 0
        weed_count = 0
        
        for r in results:
            if len(r.boxes) == 0:
                print("  ➜ Không phát hiện object nào")
            else:
                for box in r.boxes:
                    cls = int(box.cls[0])
                    cls_name = "crop" if cls == 0 else "weed"
                    conf_score = float(box.conf[0])
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    
                    if cls == 0:
                        crop_count += 1
                    else:
                        weed_count += 1
                    
                    print(f"  ➜ {cls_name.upper()} | Confidence: {conf_score:.2%} | "
                          f"BBox: [{int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}]")
        
        # Tổng kết
        if crop_count > 0 or weed_count > 0:
            print(f"  📊 Tổng: {crop_count} crops, {weed_count} weeds")
        
        # Lưu ảnh có bbox
        output_path = os.path.join(output_dir, f"pred_{os.path.basename(img_path)}")
        results[0].save(filename=output_path)
    
    print(f"\n✓ Đã lưu ảnh kết quả vào thư mục: {output_dir}/")

predict_on_test_images(model, max_images=5, conf=0.25)



print("\n" + "="*60)
print("✅ HOÀN THÀNH!")
print("="*60)

=== Kiểm tra dataset ===

TRAIN:
  ✓ 1661 images, 1661 labels

VALID:
  ✓ 580 images, 580 labels

TEST:
  ✓ 245 images, 245 labels

✓ Sử dụng data.yaml có sẵn từ Roboflow: C:\computervision\dataset\data.yaml

Nội dung data.yaml:
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['0 ridderzuring']

roboflow:
  workspace: roboflow-100
  project: grass-weeds
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/roboflow-100/grass-weeds/dataset/2

=== Bắt đầu huấn luyện ===
Ultralytics 8.3.233  Python-3.10.8 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\computervision\dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False